In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
import os

import gurobipy as gp
from gurobipy import GRB

In [3]:
for id in range(1):

    def generate_ems_data(ems_id):
        # Read and preprocess data (same as before)
        energy = pd.read_csv("../data/Final_Energy_dataset.csv", header=0)
        energy['Date'] = pd.to_datetime(energy['Date'])
        energy = energy[energy['Date'].dt.date !=pd.to_datetime('2012-02-29').date()].reset_index(drop=True)
        energy["price"] = pd.read_csv("../data/price.csv", header=0) /100
        energy.fillna(0, inplace=True)
        energy = energy.loc[35040:52559].reset_index(drop=True)

        df = pd.DataFrame({
            f'net_load_{ems_id}': energy[f'load_{ems_id}'] - energy[f'pv_{ems_id}']})
        df['load'] = energy[f'load_{ems_id}']
        df['pv'] = energy[f'pv_{ems_id}']
        df['price'] = energy["price"]
        
        return df

    data = generate_ems_data(id+1)

    bess_params = {
        'soc_init': 0,
        'soc_min': 0,
        'soc_max': 13.5,
        'eta': 1,
        's_max': 4.6,
        'bound': 100,
    }

    def build_model(data, bess_params):
        
        # Create a Gurobi model
        model = gp.Model("BESS_Optimization")

        # Extract time periods
        time_periods = data.index.to_list()

        # Precompute data
        load = data['load'].to_numpy()
        pv_gen = data['pv'].to_numpy()
        price = data['price'].to_numpy()
            
        # Decision Variables
        s_power  = model.addVars(time_periods, lb=-bess_params['s_max'], ub=bess_params['s_max'], name="s_power")
        SOC = model.addVars(time_periods, lb=bess_params['soc_min'], ub=bess_params['soc_max'], name="SOC")
        net_consumption = model.addVars(time_periods, lb=-100, ub=100, name="net_consumption")

        # ----------------------------
        # Add Constraints
        # ----------------------------
        
        # SOC Dynamics Constraints
        model.addConstrs(
        (SOC[t] == (bess_params['soc_init'] if t == time_periods[0] else SOC[time_periods[time_periods.index(t) - 1]]) + s_power[t] * bess_params['eta'] for t in time_periods),
            name="SOC_dynamics"
        )   

        model.addConstrs(
            (net_consumption[t] == load[time_periods.index(t)] - pv_gen[time_periods.index(t)] + s_power[t] for t in time_periods),
            name="NetConsumption"
        )

        # Objective function
        total_cost = gp.quicksum(price[time_periods.index(t)] * (load[time_periods.index(t)] - pv_gen[time_periods.index(t)] + s_power[t]) for t in time_periods)
        model.setObjective(total_cost, GRB.MINIMIZE)

        return model

    model = build_model(data, bess_params)

    def optimize_model(model):
        """
        Optimize the given Gurobi model.
        """
        # Set Gurobi parameters for better performance
        model.setParam('OutputFlag', 0)        # Suppress Gurobi output
        model.setParam('Threads', 8)            # Adjust based on your CPU (e.g., 8 threads for an 8-core CPU)
        model.setParam('Presolve', 2)           # Automatic presolve
        #model.setParam('Cuts', 2)               # Automatic cutting planes
        #model.setParam('Heuristics', 0.1)       # Allocate 10% of time to heuristics
        #model.Params.MIPGap = 0.01  # 1% optimality gap
        
        # Optimize the model
        model.optimize()
        return model

    model = optimize_model(model)

    def extract_results(model, data):
        """
        Extract the optimization results and add them to the data DataFrame.
        """
        # Initialize lists to store variable values
        s_power_values = []
        SOC_values = []
        NetConsumption_Optimized = []

        # Iterate through each time period to extract variable values
        for t in data.index:
            # Extract s_power
            s_power_var = model.getVarByName(f"s_power[{t}]")
            s_power_values.append(s_power_var.X)
            
            # Extract SOC
            SOC_var = model.getVarByName(f"SOC[{t}]")
            SOC_values.append(SOC_var.X)
                

            # Calculate net consumption after optimization
            net_consumption = data.loc[t, 'load'] - data.loc[t, 'pv'] + s_power_values[-1]
            NetConsumption_Optimized.append(net_consumption)
        
        # Add extracted values to the DataFrame
        data['s_power'] = s_power_values
        data['SOC'] = SOC_values
        data['NetConsumption_Optimized'] = NetConsumption_Optimized

        return data

    results = extract_results(model, data)

    def prepare_data(df):
        """
        Prepare data for plotting by renaming and calculating required columns.
        Args:
            df (pd.DataFrame): Original DataFrame with EMS data
        Returns:
            pd.DataFrame: Processed DataFrame ready for plotting
        """
        # Create new DataFrame with renamed and calculated columns
        plot_df = pd.DataFrame()
        
        # Add basic columns with renamed headers
        plot_df['soe'] = df['SOC']
        plot_df['price'] = df['price']
        plot_df['action'] = df['s_power']
        plot_df['total_load'] = df['NetConsumption_Optimized']
        
        # Calculate net load (Load + PV_Generation as PV is negative in your case)
        plot_df['net_load'] = df['load'] - df['pv']
        
        # Calculate costs
        plot_df['cost'] = -df['price'] * df['NetConsumption_Optimized']
        plot_df['total_cost'] = plot_df['cost'].cumsum()
        
        return plot_df

    results2 = prepare_data(results)
    output_path = f"outputs/baselines"
    os.makedirs(output_path, exist_ok=True)
    results2.to_csv(output_path + f"/LP_baseline_building{id+1}.csv")

In [5]:
def summarize_results(data):
    """
    Summarize the optimization results including total costs and savings.
    """
    # Total Cost after optimization
    total_cost_optimized = ((-data['price'] * data['NetConsumption_Optimized'])).sum()
    # Total Cost without optimization (No BESS actions)
    total_cost_unoptimized = (-data['price'] * (data['load'] - data['pv'])).sum()
    # Total Cost Savings
    cost_savings = total_cost_unoptimized - total_cost_optimized
    
    # Display the summary
    print("----- Optimization Summary -----")
    print(f"Total Cost without Optimization: ${total_cost_unoptimized:.2f}")
    print(f"Total Cost after Optimization:    ${total_cost_optimized:.2f}")
    print(f"Total Cost Savings:               ${cost_savings:.2f}")
    print("---------------------------------")

summarize_results(results)

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def visualize_results(data):
    """
    Visualize the optimization results including storage power, SOC, net consumption, and flexibility.
    """
    
    # Ensure there are at least 100 data points
    delta=100
    num_periods = min(delta+100, len(data))
    data_subset = data.iloc[delta:num_periods].copy()

    # Calculate Cost and Revenue per Iteration
    data_subset['Cost'] = data_subset['price'] * data_subset['NetConsumption_Optimized']
    # Calculate Cumulative Overall Costs
    data_subset['Cumulative_Cost'] = data_subset['Cost'].cumsum()
    

    # Set up the plotting environment with 5 subplots using GridSpec for better layout control
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(4, 1, height_ratios=[2, 2, 2, 2], hspace=0.4)
    
    axs = gs.subplots(sharex=True)
    
    # ----------------------------
    # Plot Storage Power (s_power)
    # ----------------------------
    axs[0].step(data_subset['Time'], data_subset['s_power'], where='post', label='Storage Power', color='blue')
    axs[0].step(data_subset['Time'], data_subset['SOC'], where='post', label='State of Charge', color='orange')
    axs[0].set_ylabel('Power (kW)')
    axs[0].set_title(f'Storage Charging/Discharging Schedule and SOC (First {num_periods} Periods)')
    axs[0].legend()
    axs[0].grid(True)
    
    # ----------------------------
    # Plot Net Consumption (NetConsumption_Optimized)
    # ----------------------------
    axs[1].step(data_subset['Time'], data_subset['NetConsumption_Optimized'], where='post', label='Net Consumption (Optimized)', color='green')
    axs[1].set_ylabel('Load (kW)')
    axs[1].set_title(f'Net Consumption Over Time (First {num_periods} Periods)')
    axs[1].legend()
    axs[1].grid(True)

    # ----------------------------
    # Plot Cost and Revenue per Iteration
    # ----------------------------
    axs[2].step(data_subset['Time'], data_subset['Cost'], where='post', label='Cost', color='red')
    axs[2].set_ylabel('Amount ($)')
    axs[2].set_title(f'Cost and Revenue per Iteration (First {num_periods} Periods)')
    axs[2].legend()
    axs[2].grid(True)
    
    # ----------------------------
    # Plot Cumulative Overall Costs
    # ----------------------------
    axs[3].plot(data_subset['Time'], data_subset['Cumulative_Cost'], label='Cumulative Cost', color='brown')
    axs[3].set_ylabel('Cumulative Cost ($)')
    axs[3].set_title(f'Cumulative Overall Costs (First {num_periods} Periods)')
    axs[3].legend()
    axs[3].grid(True)
    
    # ----------------------------
    # Formatting the x-axis
    # ----------------------------
    axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    plt.xlabel('Time')
    
    fig.tight_layout()
    plt.show()

#visualize_results(data)

----- Optimization Summary -----
Total Cost without Optimization: $-737.28
Total Cost after Optimization:    $950.05
Total Cost Savings:               $-1687.33
---------------------------------
